## Download data and source folder (**Do not Modify**)


In [1]:
!pip install gdown
import gdown
import zipfile
url = 'https://drive.google.com/uc?id=1CQdgTOUlY-TUoWZyxtVZxRthBhSuhDVi'
output = 'source.zip'
gdown.download(url, output, quiet=False)
with zipfile.ZipFile(output, "r") as zip_ref:
    zip_ref.extractall('.')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/uc?id=1CQdgTOUlY-TUoWZyxtVZxRthBhSuhDVi
To: /content/source.zip
100%|██████████| 11.8M/11.8M [00:00<00:00, 302MB/s]


## Install package for calculating FLOPS (**Do not Modify**)


In [2]:
!pip install pthflops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Necessary Dependencies
- You may import other packages if you want.


In [3]:
import os
import sys
import os.path as osp

from PIL import Image

import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as T

import glob
import matplotlib.pyplot as plt

from pthflops import count_ops

# functions that you downloaded from the first code cell.
# please use this code for seed reset, dataloaders, and test function  
from src.util import reset
# if data.py is changed, please submit that also.
from src.data import get_dataloader
from src.test import test

## Define label names, data directory, device name.


In [4]:
## DO NOT MODIFY FROM HERE
label_names = ['bug', 'electric', 'fighting', 'fire', 'flying', 'grass', 'ground', 'phychic', 'poison', 'water']
data_dir = 'data'
## DO NOT MODIFY UNTIL HERE

## You may modify device name depending on your workspace spec.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Specify the batch size as you want.
batch_size = 32

## **(TO-DO)** DEFINE YOUR `MyModel`
- Please do not change the class name. Let `MyModel` be your classifier class name. 
- Below is just simple example using ResNet18.

In [32]:
######## Define your classification model.  #######
class MyModel(nn.Module):
    def __init__(self, dim_output=len(label_names)):
        super().__init__()

        self.convnet = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 112 -> 56
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 56 -> 28
            
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 28 -> 14

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 14 -> 7
        )

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, dim_output)
        )
        
    def forward(self, img):
        B, C, H, W = img.shape
        # self.layers = []
        # out = self.features(img)
        out = self.convnet(img)
        out = out.view(-1, 512 * 7 * 7)
        out = self.classifier(out)
        return out
####################################################

* MobileNet v1

In [32]:
######## Define your classification model.  #######
# MobileNet v1 (modified)
class Depthwise(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.depthwise = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU6(),
        )

        self.pointwise = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU6()
        )
    
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class MyModel(nn.Module):
    def __init__(self, width_multiplier=1, num_classes=len(label_names), init_weights=True):
        super().__init__()
        self.init_weights=init_weights
        alpha = width_multiplier

        self.conv1 = BasicConv2d(3, int(32*alpha), 3, stride=2, padding=1)
        self.conv2 = Depthwise(int(32*alpha), int(64*alpha), stride=1)
        # down sample
        self.conv3 = nn.Sequential(
            Depthwise(int(64*alpha), int(128*alpha), stride=2),
            Depthwise(int(128*alpha), int(128*alpha), stride=1)
        )
        # down sample
        self.conv4 = nn.Sequential(
            Depthwise(int(128*alpha), int(256*alpha), stride=2),
            Depthwise(int(256*alpha), int(256*alpha), stride=1)
        )
        # down sample
        self.conv5 = nn.Sequential(
            Depthwise(int(256*alpha), int(512*alpha), stride=2),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
            # Depthwise(int(512*alpha), int(512*alpha), stride=1),
            # Depthwise(int(512*alpha), int(512*alpha), stride=1),
            # Depthwise(int(512*alpha), int(512*alpha), stride=1),
            # Depthwise(int(512*alpha), int(512*alpha), stride=1),
        )
        # down sample
        self.conv6 = nn.Sequential(
            Depthwise(int(512*alpha), int(1024*alpha), stride=2)
        )
        # down sample
        self.conv7 = nn.Sequential(
            Depthwise(int(1024*alpha), int(1024*alpha), stride=2)
        )

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.linear = nn.Linear(int(512*alpha), num_classes) # 1024 -> 512 (56*56)

        # weights initialization
        if self.init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        # x = self.conv6(x)
        # x = self.conv7(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

    # weights initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
####################################################

* EfficientNet

In [56]:

# Swish activation function
class Swish(nn.Module):
    def __init__(self):
        super().__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return x * self.sigmoid(x)

# SE Block
class SEBlock(nn.Module):
    def __init__(self, in_channels, r=4):
        super().__init__()

        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels * r),
            Swish(),
            nn.Linear(in_channels * r, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.squeeze(x)
        x = x.view(x.size(0), -1)
        x = self.excitation(x)
        x = x.view(x.size(0), x.size(1), 1, 1)
        return x

class MBConv(nn.Module):
    expand = 6
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()
        # first MBConv is not using stochastic depth
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, in_channels * MBConv.expand, 1, stride=stride, padding=0, bias=False),
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            Swish(),
            nn.Conv2d(in_channels * MBConv.expand, in_channels * MBConv.expand, kernel_size=kernel_size,
                      stride=1, padding=kernel_size//2, bias=False, groups=in_channels*MBConv.expand),
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            Swish()
        )

        self.se = SEBlock(in_channels * MBConv.expand, se_scale)

        self.project = nn.Sequential(
            nn.Conv2d(in_channels*MBConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        )

        self.shortcut = (stride == 1) and (in_channels == out_channels)

    def forward(self, x):
        # stochastic depth
        if self.training:
            if not torch.bernoulli(self.p):
                return x

        x_shortcut = x
        x_residual = self.residual(x)
        x_se = self.se(x_residual)

        x = x_se * x_residual
        x = self.project(x)

        if self.shortcut:
            x= x_shortcut + x

        return x

class SepConv(nn.Module):
    expand = 1
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()
        # first SepConv is not using stochastic depth
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels * SepConv.expand, in_channels * SepConv.expand, kernel_size=kernel_size,
                      stride=1, padding=kernel_size//2, bias=False, groups=in_channels*SepConv.expand),
            nn.BatchNorm2d(in_channels * SepConv.expand, momentum=0.99, eps=1e-3),
            Swish()
        )

        self.se = SEBlock(in_channels * SepConv.expand, se_scale)

        self.project = nn.Sequential(
            nn.Conv2d(in_channels*SepConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        )

        self.shortcut = (stride == 1) and (in_channels == out_channels)

    def forward(self, x):
        # stochastic depth
        if self.training:
            if not torch.bernoulli(self.p):
                return x

        x_shortcut = x
        x_residual = self.residual(x)
        x_se = self.se(x_residual)

        x = x_se * x_residual
        x = self.project(x)

        if self.shortcut:
            x= x_shortcut + x

        return x

class MyModel(nn.Module):
    def __init__(self, num_classes=len(label_names), width_coef=1., depth_coef=1., scale=1., dropout=0.2, se_scale=4, stochastic_depth=False, p=0.5):
        super().__init__()
        channels = [32, 16, 24, 40, 80, 112, 192, 320, 1280]
        repeats = [1, 2, 2, 3, 3, 4, 1]
        strides = [1, 2, 2, 2, 1, 2, 1]
        kernel_size = [3, 3, 5, 3, 5, 5, 3]
        depth = depth_coef
        width = width_coef

        channels = [int(x*width) for x in channels]
        repeats = [int(x*depth) for x in repeats]

        # stochastic depth
        if stochastic_depth:
            self.p = p
            self.step = (1 - 0.5) / (sum(repeats) - 1)
        else:
            self.p = 1
            self.step = 0


        # efficient net
        self.upsample = nn.Upsample(scale_factor=scale, mode='bilinear', align_corners=False)

        self.stage1 = nn.Sequential(
            nn.Conv2d(3, channels[0],3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(channels[0], momentum=0.99, eps=1e-3)
        )

        self.stage2 = self._make_Block(SepConv, repeats[0], channels[0], channels[1], kernel_size[0], strides[0], se_scale)

        self.stage3 = self._make_Block(MBConv, repeats[1], channels[1], channels[2], kernel_size[1], strides[1], se_scale)

        self.stage4 = self._make_Block(MBConv, repeats[2], channels[2], channels[3], kernel_size[2], strides[2], se_scale)

        self.stage5 = self._make_Block(MBConv, repeats[3], channels[3], channels[4], kernel_size[3], strides[3], se_scale)

        self.stage6 = self._make_Block(MBConv, repeats[4], channels[4], channels[5], kernel_size[4], strides[4], se_scale)

        self.stage7 = self._make_Block(MBConv, repeats[5], channels[5], channels[6], kernel_size[5], strides[5], se_scale)

        self.stage8 = self._make_Block(MBConv, repeats[6], channels[6], channels[7], kernel_size[6], strides[6], se_scale)

        self.stage9 = nn.Sequential(
            nn.Conv2d(channels[7], channels[8], 1, stride=1, bias=False),
            nn.BatchNorm2d(channels[8], momentum=0.99, eps=1e-3),
            Swish()
        ) 

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(p=dropout)
        self.linear = nn.Linear(channels[8], num_classes)

    def forward(self, x):
        x = self.upsample(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)
        x = self.stage6(x)
        x = self.stage7(x)
        x = self.stage8(x)
        x = self.stage9(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.linear(x)
        return x


    def _make_Block(self, block, repeats, in_channels, out_channels, kernel_size, stride, se_scale):
        strides = [stride] + [1] * (repeats - 1)
        layers = []
        for stride in strides:
            layers.append(block(in_channels, out_channels, kernel_size, stride, se_scale, self.p))
            in_channels = out_channels
            self.p -= self.step

        return nn.Sequential(*layers)


def efficientnet_b0(num_classes=len(label_names)):
    return MyModel(num_classes=num_classes, width_coef=1.0, depth_coef=1.0, scale=1.0,dropout=0.2, se_scale=4)

def efficientnet_b1(num_classes=len(label_names)):
    return MyModel(num_classes=num_classes, width_coef=1.0, depth_coef=1.1, scale=240/224, dropout=0.2, se_scale=4)

def efficientnet_b2(num_classes=len(label_names)):
    return MyModel(num_classes=num_classes, width_coef=1.1, depth_coef=1.2, scale=260/224., dropout=0.3, se_scale=4)

def efficientnet_b3(num_classes=len(label_names)):
    return MyModel(num_classes=num_classes, width_coef=1.2, depth_coef=1.4, scale=300/224, dropout=0.3, se_scale=4)

def efficientnet_b4(num_classes=len(label_names)):
    return MyModel(num_classes=num_classes, width_coef=1.4, depth_coef=1.8, scale=380/224, dropout=0.4, se_scale=4)

def efficientnet_b5(num_classes=len(label_names)):
    return MyModel(num_classes=num_classes, width_coef=1.6, depth_coef=2.2, scale=456/224, dropout=0.4, se_scale=4)

def efficientnet_b6(num_classes=len(label_names)):
    return MyModel(num_classes=num_classes, width_coef=1.8, depth_coef=2.6, scale=528/224, dropout=0.5, se_scale=4)

def efficientnet_b7(num_classes=len(label_names)):
    return MyModel(num_classes=num_classes, width_coef=2.0, depth_coef=3.1, scale=600/224, dropout=0.5, se_scale=4)

* MobileNet v2

In [12]:
# MobileNet v2
# 첫번째 layer에서 사용될 convolution 함수
def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )

# inverted bottleneck layer 바로 다음에 나오는 convolution에 사용될 함수
def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )

# channel수를 무조건 8로 나누어 떨어지게 만드는 함수
def make_divisible(x, divisible_by=8):
    import numpy as np
    return int(np.ceil(x * 1. / divisible_by) * divisible_by)


class InvertedResidual(nn.Module):

    def __init__(self, inp, oup, stride, expand_ratio):
        super().__init__()
        self.stride = stride
        assert stride in [1,2]

        hidden_dim = int(inp * expand_ratio)  # expansion channel
        self.use_res_connect = self.stride == 1 and inp == oup  # skip connection이 가능한지 확인 True or False
        '''
        self.stride == 1 ----> 연산 전 후의 feature_map size가 같다는 의미
        inp == oup ----> 채널수도 동일하게 유지된다는 의미
        즉 skip connection 가능
        '''
        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # 확장시킬 필요가 없기 때문에 바로 depth wise conv
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup)
            )
        else:
            self.conv = nn.Sequential(
                # pw(확장)
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear(축소)
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup)
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)  # skip connection (element wise sum)
        else:
            return self.conv(x)


class MyModel(nn.Module):

    def __init__(self, n_class=len(label_names), input_size=224, width_mult=1.):
        super().__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280

        interverted_residual_setting = [
            # t, c, n, s
            # t : expand ratio
            # c : channel
            # n : Number of iterations
            # s : stride
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1]
        ]

        # building first layer
        assert input_size % 32 == 0
        # input_channel = make_divisible(input_channel * width_mult)
        self.last_channel = make_divisible(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, 2)]  # feature들을 담을 리스트에 first layer 추가

        # building inverted residual blocks
        for t, c, n, s in interverted_residual_setting:
            output_channel = make_divisible(c * width_mult) if t > 1 else c
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, s, t))
                else:
                    self.features.append(block(input_channel, output_channel, 1, t))  # 반복되는 부분에서 skip connection 가능
                input_channel = output_channel
        # building last several layers
        self.features.append(conv_1x1_bn(input_channel, self.last_channel))  # (batch, 320, 7, 7) -> (batch, 1280, 7, 7)
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # Average pooling layer
        self.avg = nn.AvgPool2d(7, 7)
        # building classifier
        self.classifier = nn.Linear(self.last_channel, n_class)

        self._initialize_weights()

    def forward(self, x):
        # pdb.set_trace()
        x = self.features(x)
        x = self.avg(x)
        x = x.view(-1, self.last_channel)
        x = self.classifier(x)
        return x

    # 초기 weight 설정
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, np.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

## **(Optional)** Make your own loss function.
- You may change `criterion` as you want. 
- But make your custom loss function work without changing below lines, which starts from `model.train()`


In [6]:
def train(model, optimizer, sample):
    ### You may define your own loss function.###
    criterion = nn.CrossEntropyLoss()
    #############################################

    ### make your code work without changing below lines ###
    model.train()    
    
    input = sample['img'].float().to(device)
    label = sample['label'].long().to(device)

    pred = model(input)

    loss = criterion(pred, label)

    num_correct = torch.sum(torch.argmax(pred, dim=-1)==label)

    # wandb.watch(model, criterion, log="all", log_freq=1) # to be deleted

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), num_correct.item()
    ##########################################################

## **(TO-DO)** DEFINE YOUR `get_optimizer`
- Please do not change the function name. Let `get_optimizer` be your classifier class name. 
- This function should return the optimizer for optimizing model parameters properly


In [7]:
######## Define your own function for optimizer.  #######
def get_optimizer(model, lr=0.01):
    return optim.Adam(model.parameters(), lr=lr)
##########################################################

## Repeat Training with different 10 random seeds.

*   Do not change `max_epoch` and `num_seeds`.
*   You  may change lines for `MyModel` and `get_optimizer` part if they require additional inputs.


In [11]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!wandb login

wandb: Currently logged in as: ray_park. Use `wandb login --relogin` to force relogin


In [13]:
import wandb

In [7]:
!cat ./src/data.py

import os
import os.path as osp
import random
import numpy as np

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T


def random_split(data_dir, label_names, div_rate=8):
    image_fps = []
    label_idx = []
        
    for l_idx, l_name in enumerate(label_names):
        curr_label = l_idx
        dir_fp = osp.join(data_dir, l_name)
        for img_fp in sorted(os.listdir(dir_fp)):
            image_fps.append(osp.join(dir_fp, img_fp))
            label_idx.append(curr_label)
    
    num_total = len(image_fps)
    num_test = num_total // div_rate
    num_train = num_total - num_test

    all_indices = np.arange(len(image_fps))
    
    random.shuffle(all_indices)
    test_indices = all_indices[:num_test]
    train_indices = all_indices[num_test:]

    train_image_fps = [image_fps[idx] for idx in train_indices]
    train_label_idx = [label_idx[idx] for idx in train_indices]

    test_image_fps = [image_fps[idx] for idx in test_

In [28]:
# data.py (colab 파일을 수정해도 반영되지 않아 직접 올립니다.)
import os
import os.path as osp
import random
import numpy as np

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T


def random_split(data_dir, label_names, div_rate=8):
    image_fps = []
    label_idx = []
        
    for l_idx, l_name in enumerate(label_names):
        curr_label = l_idx
        dir_fp = osp.join(data_dir, l_name)
        for img_fp in sorted(os.listdir(dir_fp)):
            image_fps.append(osp.join(dir_fp, img_fp))
            label_idx.append(curr_label)
    
    num_total = len(image_fps)
    num_test = num_total // div_rate
    num_train = num_total - num_test

    all_indices = np.arange(len(image_fps))
    
    random.shuffle(all_indices)
    test_indices = all_indices[:num_test]
    train_indices = all_indices[num_test:]

    train_image_fps = [image_fps[idx] for idx in train_indices]
    train_label_idx = [label_idx[idx] for idx in train_indices]

    test_image_fps = [image_fps[idx] for idx in test_indices]
    test_label_idx = [label_idx[idx] for idx in test_indices]
    
    return train_image_fps, train_label_idx, test_image_fps, test_label_idx


class PokemonDataset(Dataset):
    def __init__(self, image_fps, label_idx, is_train):
        self.image_fps = image_fps
        self.label_idx = label_idx 
        self.is_train = is_train

        if self.is_train:
            self.transform = T.Compose([T.AutoAugment(),
                                        T.Resize((56, 56)), 
                                        T.ToTensor()])
        else:
            self.transform = T.Compose([T.Resize((56, 56)),
                                        T.ToTensor()])

    def __len__(self):
        return len(self.image_fps)

    def __getitem__(self, idx):
        curr_image_fp = self.image_fps[idx]
        curr_label_idx = self.label_idx[idx]

        sample = dict()
        sample['img'] = self.transform(Image.open(curr_image_fp))
        sample['label'] = curr_label_idx

        return sample

def get_dataloader(data_dir, label_names, batch_size):
    train_image_fps, train_label_idx, test_image_fps, test_label_idx = random_split(data_dir, label_names)
    train_dataset = PokemonDataset(train_image_fps, train_label_idx, True)
    test_dataset = PokemonDataset(test_image_fps, test_label_idx, False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader, train_dataset, test_dataset

In [33]:
### Do not change below parameters ###
max_epoch = 5
num_seeds = 10
### Do not change above parameters ###


### Make your code work without changing below lines except for "optimizer" part ###
# wandb.init(project="AIToolkit_Assignment04", entity="ray_park") # to be deleted
print('batch size: ', batch_size) # to be deleted
total_best_accu = []

for seed in range(num_seeds):
    # do not erase this random seed initialization part
    reset(seed)

    # get dataloader by spliting train/test data randomly.
    train_loader, test_loader, train_dataset, test_dataset = get_dataloader(data_dir, label_names, batch_size)
    print('img size: ', next(iter(train_loader))['img'].shape) # to be deleted

    # you may change below line for model definition if your "MyModel" requires more inputs.
    model = MyModel().to(device)

    # you may change below line for optimizer if your "get optimizer" requires more inputs.
    optimizer = get_optimizer(model)

    ###### do not change below lines. Make your code work with below lines. ####
    best_accu = -100
    for epoch in range(max_epoch):
        avg_tr_loss = 0.0
        avg_tr_correct = 0.0
        for sample in train_loader:
            tr_loss, tr_correct = train(model, optimizer, sample)

            avg_tr_loss += tr_loss / len(train_loader)
            avg_tr_correct += tr_correct / len(train_dataset)

        avg_te_correct = 0.0
        for sample in test_loader:
            te_correct = test(model, sample, device)
            avg_te_correct += te_correct / len(test_dataset)
    
        best_accu = max(avg_te_correct, best_accu)
        print('train acc: ', avg_tr_correct) # to be deleted

        # wandb.log({"Epoch": epoch, "avg_tr_correct": avg_tr_correct, "avg_te_correct": avg_te_correct}) # to be deleted

    print('<<<<<[SEED {}] BEST ACCU : {}>>>>>'.format(seed, best_accu))    
    total_best_accu.append(best_accu)
    if seed < num_seeds-1:
        del model, optimizer
    ############################################################################   
# wandb.finish() # to be deleted

batch size:  32
img size:  torch.Size([32, 3, 56, 56])
train acc:  0.17653131452167947
train acc:  0.25120440467997274
train acc:  0.27735719201651776
train acc:  0.3303509979353064
train acc:  0.35684790089470075
<<<<<[SEED 0] BEST ACCU : 0.37590361445783127>>>>>
img size:  torch.Size([32, 3, 56, 56])
train acc:  0.19442532690984193
train acc:  0.23916035788024803
train acc:  0.2828630419821062
train acc:  0.31211286992429477
train acc:  0.3303509979353063
<<<<<[SEED 1] BEST ACCU : 0.39036144578313253>>>>>
img size:  torch.Size([32, 3, 56, 56])
train acc:  0.18960770818995207
train acc:  0.2556779077770134
train acc:  0.30591878871300776
train acc:  0.31280110116999327
train acc:  0.3640743289745356
<<<<<[SEED 2] BEST ACCU : 0.472289156626506>>>>>
img size:  torch.Size([32, 3, 56, 56])
train acc:  0.20991052993805945
train acc:  0.2680660701995874
train acc:  0.31211286992429466
train acc:  0.3317274604267035
train acc:  0.3589125946317964
<<<<<[SEED 3] BEST ACCU : 0.4409638554216868>

## Calculate your grade **(Do not Modify)**
- Grade is calculated based on your `average best accuracy` and `FLOPS`.
- Higher accuracy and lower flops make your grade better.

In [34]:
###### do not change below lines #####
num_ops = count_ops(model, torch.rand(1, 3, 56, 56).to(device), verbose=False)

mean_accu = np.mean(total_best_accu)
accu_thres = 0.75
accu_point = min(1, np.exp(-2*(accu_thres-mean_accu)))

flops = num_ops[0]
flops_thres = 2e8
flop_disadvantage = flops_thres / (flops_thres + flops)

print('*'*50)
print('Mean Accuracy :', mean_accu)
print('Accuracy Point:', accu_point)
print('*'*50)
print('Flops: ', flops)
print('Flops Advantage: ', flop_disadvantage)
print('*'*50)

point = accu_point * flop_disadvantage
print('Total Points : ', point)

threshold = 0.5
max_point = 50

if point > threshold:
    grade = max_point
else:
    grade = np.exp(-2*(threshold - point)) * max_point

print('YOUR grade is {} point'.format(grade))

Input size: (1, 3, 56, 56)
19,662,730 FLOPs or approx. 0.02 GFLOPs
**************************************************
Mean Accuracy : 0.45373493975903606
Accuracy Point: 0.5529265426880182
**************************************************
Flops:  19662730
Flops Advantage:  0.910486726628591
**************************************************
Total Points :  0.5034322779180775
YOUR grade is 50 point
